In [1]:
import numpy as np
import pandas as pd
import streamlit as st
from datetime import datetime, timedelta
import os
import gspread
from google.oauth2 import service_account
from gspread_dataframe import set_with_dataframe
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from configs import *
import requests
from functionssb import *

In [2]:
gs_connection = connect_to_gs(st.secrets["gcp_service_account"])

In [3]:
drinks = fetch_drinks_data(gs_connection, drinks_table, prod_google_sheet_key, ['event','drink_size', 'start_time', 'end_time'])

In [4]:
drinks

,event,nominator_name,drinker_name,drink_type,nomination_created_date,nomination_deadline_date,nomination_completed_date,quantity,drink_size,start_time,end_time
0,1,auto,Jonno Frame,last place,1692075600,1692748799,1692725340,,330.0,4.25,12.04
1,1,Devlin Sharp,Connor McDonald,nomination,1692075600,1692748799,1692110760,,330.0,5.38,11.04
2,1,Devlin Sharp,Tijil Jugadoe,nomination,1692075600,1692748799,1692289560,,330.0,0.00,10.88
3,1,Devlin Sharp,Niklas Wietzorrek,nomination,1692075600,1692748799,1692462600,,330.0,1.03,7.38
4,2,auto,Tijil Jugadoe,red card,1692680400,1693353599,1692688020,,440.0,0.00,8.70
5,2,auto,Rhys Floyd,red card,1692680400,1693353599,1693330440,,440.0,1.37,45.90
6,2,auto,Lawrence Fowle,own goal,1692680400,1693353599,1692960660,,425.0,4.82,10.16
7,2,auto,Divyam Dixit,missed pen,1692680400,1693353599,1693319340,,330.0,NaN,NaN
8,2,auto,Alex Wietzorrek,missed pen,1692680400,1693353599,1692806460,,330.0,2.20,6.40
9,2,auto,Jason Perry,missed pen,1692680400,1693353599,1692688260,,330.0,1.90,7.12


In [7]:
df = drinks
df["quantity"] = 1
df = df.iloc[:, [2, 5, 6, 7]]
df.rename(columns={"drinker_name": "Name"}, inplace=True)
df


/var/folders/c7/dcyx4ss130ldp4lp9ph0v16c0000gn/T/ipykernel_1456/2993913015.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"drinker_name": "Name"}, inplace=True)


,Name,nomination_deadline_date,nomination_completed_date,quantity
0,Jonno Frame,1692748799,1692725340,1
1,Connor McDonald,1692748799,1692110760,1
2,Tijil Jugadoe,1692748799,1692289560,1
3,Niklas Wietzorrek,1692748799,1692462600,1
4,Tijil Jugadoe,1693353599,1692688020,1
5,Rhys Floyd,1693353599,1693330440,1
6,Lawrence Fowle,1693353599,1692960660,1
7,Divyam Dixit,1693353599,1693319340,1
8,Alex Wietzorrek,1693353599,1692806460,1
9,Jason Perry,1693353599,1692688260,1


In [8]:
time1 = 'Not Completed'
time2 = '1693864799'

In [11]:
x = int(time2)
date_obj = datetime.fromtimestamp(x)
date_obj.strftime('%d %b')

'04 Sep'

In [ ]:
try:
    timestamp = int(date_str)  # Assuming date_str is a Unix timestamp in string format
    date_obj = datetime.fromtimestamp(timestamp)
    return date_obj.strftime('%d %b')
except ValueError:
    try:
        date_obj = datetime.strptime(date_str, '%d/%m/%y %H:%M')
        return date_obj.strftime('%d %b')
    except ValueError:
        try:
            date_obj = datetime.strptime(date_str, '%d/%m/%y')
            return date_obj.strftime('%d %b')
        except ValueError:
            return "Not Completed"

In [ ]:
def create_unix(date_str):
    try:
        timestamp = int(date_str)
        return timestamp
    except ValueError:
        return "Not Completed"

In [14]:
df["nomination_completed_date"] = df["nomination_completed_date"].apply(lambda x: format_date(x))
df["nomination_deadline_date"] = df["nomination_deadline_date"].apply(lambda x: format_date(x))
df

/var/folders/c7/dcyx4ss130ldp4lp9ph0v16c0000gn/T/ipykernel_51732/3995295841.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["nomination_completed_date"] = df["nomination_completed_date"].apply(lambda x: format_date(x))
/var/folders/c7/dcyx4ss130ldp4lp9ph0v16c0000gn/T/ipykernel_51732/3995295841.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["nomination_deadline_date"] = df["nomination_deadline_date"].apply(lambda x: format_date(x))


,Name,nomination_deadline_date,nomination_completed_date,quantity
0,Jonno Frame,Not Completed,Not Completed,1
1,Connor McDonald,Not Completed,Not Completed,1
2,Tijil Jugadoe,Not Completed,Not Completed,1
3,Niklas Wietzorrek,Not Completed,Not Completed,1
4,Tijil Jugadoe,Not Completed,Not Completed,1
5,Rhys Floyd,Not Completed,Not Completed,1
6,Lawrence Fowle,Not Completed,Not Completed,1
7,Divyam Dixit,Not Completed,Not Completed,1
8,Alex Wietzorrek,Not Completed,Not Completed,1
9,Jason Perry,Not Completed,Not Completed,1


In [5]:
df = categories(drinks)
df

/Users/connormcdonald/Desktop/GitHub/fpl-v2/functionssb.py:507: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"drinker_name": "Name"}, inplace=True)
/Users/connormcdonald/Desktop/GitHub/fpl-v2/functionssb.py:510: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["nomination_completed_date"] = pd.to_datetime(df["nomination_completed_date"], errors="coerce")
/Users/connormcdonald/Desktop/GitHub/fpl-v2/functionssb.py:511: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

,Name,nomination_deadline_date,nomination_completed_date,quantity,Category
0,Jonno Frame,2023-08-22 23:59:00,2023-08-22 19:29:00,1,Completed
1,Connor McDonald,2023-08-22 23:59:00,2023-08-15 16:46:00,1,Completed
2,Tijil Jugadoe,2023-08-22 23:59:00,2023-08-17 18:26:00,1,Completed
3,Niklas Wietzorrek,2023-08-22 23:59:00,2023-08-19 18:30:00,1,Completed
4,Tijil Jugadoe,2023-08-29 23:59:00,2023-08-22 09:07:00,1,Completed
5,Rhys Floyd,2023-08-29 23:59:00,NaT,1,Outstanding
6,Lawrence Fowle,2023-08-29 23:59:00,2023-08-25 12:51:00,1,Completed
7,Divyam Dixit,2023-08-29 23:59:00,2023-08-29 16:29:00,1,Completed
8,Alex Wietzorrek,2023-08-29 23:59:00,2023-08-23 18:01:00,1,Completed
9,Jason Perry,2023-08-29 23:59:00,2023-08-22 09:11:00,1,Completed


In [6]:
df.dtypes

Name                                 object
nomination_deadline_date     datetime64[ns]
nomination_completed_date    datetime64[ns]
quantity                              int64
Category                             object
dtype: object

In [7]:
a = df.iloc[33]["nomination_deadline_date"]

In [8]:
b = df.iloc[33]["nomination_completed_date"]

In [9]:
a>b

True